In [1]:
from flask import Flask, json, request
from Client_Encryption import *
from urllib.parse import urlparse
import requests
import ast
from cryptography.fernet import Fernet
from Client_Model import *
import os

In [ ]:
api = Flask(__name__)
from flask import session, url_for, redirect

@api.route('/register', methods=['GET'])
def register():
    
    o = urlparse(request.base_url)
    PARAMS = {'url':o.hostname, 'port':o.port}
    r = requests.get(url = 'http://127.0.0.1:10011/register', params = PARAMS)
    
    #print(r, r.status_code, r.reason, r.text)
    if r.status_code == 200:
        return "Status OK sent !"
    
    return "Failed Status"

@api.route('/active_test', methods=['GET'])
def active_test():
    ack = {'client_ack': '1'}
    return str(ack)

@api.route('/downloadmodel', methods=['POST'])
def get_agg_model():
    if request.method == 'POST':
        
        file = request.files['model'].read()
        order_file = request.files['order_file'].read()
        data = request.files['json'].read()

        data = ast.literal_eval(data.decode("utf-8"))
        fname = data['fname']
        order_file_name = data['order_file']
        
        with open('filekey.key', 'rb') as filekey:
            key = filekey.read()
        
        fernet = Fernet(key)
        order_file = fernet.decrypt(order_file)
        print(order_file)
        
        wfile = open(fname, 'wb+')
        wfile.write(file)
        wfile.close()
        
        wfile = open(order_file_name, 'wb+')
        wfile.write(order_file)
        wfile.close()
        
        decrypt_file(fname, order_file_name)        
        os.remove(order_file_name)
        return "Model received!"
    
    else:
        return "No file received!"
    
@api.route('/sendmodel', methods=['GET'])
def send_model():
    client_train()
    print("Training over")
    order_file_path, file_name = encrypt_file('agg_model.npy')
    
    with open('filekey.key', 'rb') as filekey:
        key = filekey.read()
        
    fernet = Fernet(key)
    data = fernet.encrypt(data = open(order_file_path, 'rb').read())
    with open(order_file_path, 'wb+') as w:
        w.write(data)
    
    file = open(file_name, 'rb')
    order_file = open(order_file_path, 'rb')
    
    o = urlparse(request.base_url)
    data = {'fname':'agg_model.npy', 'url':o.hostname, 'port':o.port, 'order_file':'order_file'}
    files = {
        'json': ('json_data', json.dumps(data), 'application/json'),
        'model': ('agg_model.npy', file, 'application/octet-stream'),
        'order_file': ('order_file', order_file, 'application/octet-stream')
    }

    req = requests.post(url='http://127.0.0.1:10011/client_model', files=files)
    
    file.close()
    order_file.close()
    os.remove(order_file_path)
    os.remove(file_name)
    return "Model sent !"
    
if __name__ == '__main__':
    api.secret_key = '2458867997' ##set to random
    api.config['SESSION_TYPE'] = 'filesystem'
    api.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jul/2022 23:37:47] "GET / HTTP/1.1" 404 -


Processing data...
10000 test samples
Agg weights exists...
Loading weights...
469/469 [==============================] - 38s 81ms/step - loss: 0.0240 - accuracy: 0.9924 - val_loss: 0.0323 - val_accuracy: 0.9912
Test loss: 0.03229633718729019
Test accuracy: 0.9911999702453613
Local model update written to local storage!
Training over


C:\Users\pk15425\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
127.0.0.1 - - [14/Jul/2022 23:38:38] "GET /sendmodel HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2022 23:38:48] "GET /active_test HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2022 23:38:49] "POST /downloadmodel HTTP/1.1" 200 -


b'{"0": 3, "1": 1, "2": 2, "3": 6, "4": 5, "5": 7, "6": 0, "7": 4}'
